## 一、数据预处理

In [0]:
# 导入依赖包
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential, datasets, losses, optimizers

In [2]:
# 加载数据集
(x, y), (x_test, y_test) = datasets.cifar10.load_data()
print(x.shape, y.shape, x_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [3]:
# 数据预处理并转化为Datasets

def preprocess(x, y):
    """
    预处理函数
    """
    x = tf.cast(x, dtype=tf.float32) / 255
    y = tf.cast(y, tf.int32)
    # [b, 1] => [b]
    y = tf.squeeze(y)
    # [b, 10]
    y = tf.one_hot(y, depth=10)
    return x,y

epoch_num = 50
batch_size = 128
learning_rate = 1e-3

# 转化为Dataset数据集
train_db = tf.data.Dataset.from_tensor_slices((x, y))
# map --> shuffle ---> batch: 先map，后batch，否则会出现ValueError: Cannot take the length of shape with unknown rank.
train_db = train_db.map(preprocess).shuffle(50000).batch(batch_size)

test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_db = test_db.map(preprocess).batch(batch_size)


sample_train = next(iter(train_db))
print('batch_train:', sample_train[0].shape, sample_train[1].shape)
sample_test = next(iter(test_db))
print('batch_test:', sample_test[0].shape, sample_test[1].shape)

batch_train: (128, 32, 32, 3) (128, 10)
batch_test: (128, 32, 32, 3) (128, 10)


## 二、实现Basic Block
![Basic Block](https://freeshow.oss-cn-beijing.aliyuncs.com/blog/20200420101359.png)

In [0]:
class BasicBlock(layers.Layer):

  def __init__(self, filter_num, stride=1):
    super(BasicBlock, self).__init__()

    # F(x)
    self.conv1 = layers.Conv2D(filter_num, (3, 3), strides=stride, padding='same')
    self.bn1 = layers.BatchNormalization()
    self.relu = layers.Activation('relu')

    self.conv2 = layers.Conv2D(filter_num, (3, 3), strides=1, padding='same')
    self.bn2 = layers.BatchNormalization()

    # identity, 如果stride不等于1，说明F(x)降维了，则shutcut也需要降维来保持和F(x)具有相同的维度。 
    if stride != 1:
        self.downsample = Sequential()
        self.downsample.add(layers.Conv2D(filter_num, (1, 1), strides=stride))
    else:
        self.downsample = lambda x:x

  def call(self, inputs, training=None):
    # [b, h, w, c]
    out = self.conv1(inputs)
    out = self.bn1(out,training=training)
    out = self.relu(out)

    out = self.conv2(out)
    out = self.bn2(out,training=training)

    identity = self.downsample(inputs)

    output = layers.add([out, identity])
    output = tf.nn.relu(output)

    return output

## 三、实现 Res Block
![ResNet18, image found on Google images.
](https://freeshow.oss-cn-beijing.aliyuncs.com/blog/20200420171051.png)

如上图 ResNet18 包含一个数据处理层，4个Res Block层(每种颜色代表一个Res Block,每个Res Block包含两个Basic Block)和一个输出层，每个弧线代表一个Basic Block，每个Res Block的第一个Basic Block用虚弧线表示feathure map减半，每个Basic Block包含两个Conv, 所以 1+4*4+1=18层

In [0]:
class ResNet(keras.Model):
  def __init__(self, layer_dims, num_classes=10): # [2, 2, 2, 2]
    super(ResNet, self).__init__()

    # 第一层，数据处理层
    self.stem = Sequential([layers.Conv2D(64, (3, 3), strides=(1, 1)),
                            layers.BatchNormalization(),
                            layers.Activation('relu'),
                            layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1), padding='same')
                            ])
    # 第1个Res Block
    self.layer1 = self.build_resblock(64,  layer_dims[0])
    # 第2个Res Block
    self.layer2 = self.build_resblock(128, layer_dims[1], stride=2)
    # 第3个Res Block
    self.layer3 = self.build_resblock(256, layer_dims[2], stride=2)
    # 第4个Res Block, [b, 512, h, w]
    self.layer4 = self.build_resblock(512, layer_dims[3], stride=2)

    # 最后一层，输出层，使用GlobalAveragePooling2D代替全连接层(batch_size, channels)，即[b, 512]
    self.avgpool = layers.GlobalAveragePooling2D()
    self.fc = layers.Dense(num_classes)

  def call(self, inputs, training=None):
    x = self.stem(inputs,training=training)

    x = self.layer1(x,training=training)
    x = self.layer2(x,training=training)
    x = self.layer3(x,training=training)
    x = self.layer4(x,training=training)

    # [b, c]
    x = self.avgpool(x)
    # [b, 10]
    x = self.fc(x)

    return x

  def build_resblock(self, filter_num, blocks, stride=1):
    """
    建立Res Block
    """
    res_blocks = Sequential()
    # may down sample, 第一层Basic Block一般需要降维
    res_blocks.add(BasicBlock(filter_num, stride))

    # 后面的Res Block一般不需要降维
    for _ in range(1, blocks):
        res_blocks.add(BasicBlock(filter_num, stride=1))

    return res_blocks

## 四、定义ResNet

In [0]:
def resnet18():
  """
  ResNet18
  """
  return ResNet([2, 2, 2, 2])

def resnet34():
  """
  ResNet34
  """
  return ResNet([3, 4, 6, 3])

## 五、创建网络

In [7]:
model = resnet18()
model.build(input_shape=(None, 32, 32, 3))
model.summary()

Model: "res_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      multiple                  2048      
_________________________________________________________________
sequential_1 (Sequential)    multiple                  148736    
_________________________________________________________________
sequential_2 (Sequential)    multiple                  526976    
_________________________________________________________________
sequential_4 (Sequential)    multiple                  2102528   
_________________________________________________________________
sequential_6 (Sequential)    multiple                  8399360   
_________________________________________________________________
global_average_pooling2d (Gl multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  5130

## 六、模型训练

In [0]:
model.compile(loss=losses.CategoricalCrossentropy(from_logits=True), optimizer=optimizers.Adam(learning_rate=learning_rate), metrics=["accuracy"])
model.fit(train_db, epochs=epoch_num, validation_data=test_db, validation_freq=10)